In [13]:
from langchain_groq import ChatGroq

In [14]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [15]:
from langchain.document_loaders import WebBaseLoader

In [16]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/?jobSearch=true&jsOffset=0&jsSort=posting_start_date&jsLanguage=en")
page_data = loader.load().pop().page_content
print(page_data)

Nike Careers Home

Section 1: Move the World. Welcome to Nike Careers.Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsMove the WorldWelcome to Nike CareersSearch JobsScrollSearch JobsFin

In [17]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [18]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
print(json_res)

[{'role': 'Software Engineer', 'experience': '3-5 years', 'skills': ['Java', 'Python', 'Cloud Computing'], 'description': 'Design and develop software applications for our e-commerce platform.'}, {'role': 'Marketing Manager', 'experience': '5-7 years', 'skills': ['Marketing Strategy', 'Brand Management', 'Data Analysis'], 'description': 'Develop and execute marketing campaigns to drive brand awareness and sales.'}, {'role': 'Data Scientist', 'experience': '2-4 years', 'skills': ['Machine Learning', 'Data Visualization', 'Statistics'], 'description': 'Analyze customer data to inform business decisions and drive growth.'}]


In [19]:
type(json_res)

list

In [20]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [21]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [29]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}]]

In [28]:

job['skills']

['Java', 'Python', 'Cloud Computing']

In [27]:
job = json_res[0]  # Access the first item in the list
if 'skills' in job:
    skills = job['skills']
    print(skills)
else:
    print("No 'skills' key found in the job object")


['Java', 'Python', 'Cloud Computing']


In [30]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Sahil, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Sahil, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Engineering Solutions for Your E-commerce Platform

Dear Hiring Manager,

I came across your job posting for a Software Engineer with expertise in Java, Python, and Cloud Computing. I'd like to introduce you to AtliQ, an AI & Software Consulting company that has been empowering enterprises with tailored solutions for seamless business process integration.

Our team of expert software engineers has extensive experience in designing and developing software applications for e-commerce platforms. We've successfully delivered numerous projects that have resulted in scalability, process optimization, cost reduction, and heightened overall efficiency for our clients.

I'd like to highlight a few of our relevant portfolio pieces that demonstrate our capabilities in Java, Python, and Cloud Computing:

- Our Java expertise: https://example.com/java-portfolio
- Our Python expertise: https://example.com/python-portfolio and https://example.com/ml-python-portfolio
- Our Dev